In [ ]:
%pip install -U chromadb langchain accelerate transformers bitsandbytes

In [ ]:
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

vector_db_path = '/content/drive/MyDrive/Colab Notebooks/RAG/Vector DB'

hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma(collection_name="user_docs", embedding_function=hf_embed, persist_directory=vector_db_path)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:

def get_similar_docs(question, similar_doc_count):
  return db.similarity_search(question, k=similar_doc_count)

for doc in get_similar_docs("En qué meses me he gastado más de $100.000 en Luz?", 2):
  print(doc.page_content)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

def build_qa_chain():
  torch.cuda.empty_cache()
  model_name = 'databricks/dolly-v2-3b'
  """  quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
  )"""
  #model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto", quantization_config=quantization_config)

  instruct_pipeline = pipeline(model=model_name, trust_remote_code=True, device_map="auto",
                                return_full_text=True, max_new_tokens=256, top_p=0.95, top_k=50)

  template = """Debajo hay una instruccion que describe una tarea. Escribe una respuesta que completa apropiadamente la peticion.

  Instruccion:
  Eres un asistente personal (chatbot) teniendo una conversacion con un humano y tu trabajo es ayudar a los usuarios suministrandoles la informacion mas acertada y detallada sobre sus documentos basandote en sus preguntas.
  Solo utiliza informacion en los siguientes parrafos para responder la pregunta al final. Explica la respuesta referenciando estos parrafos. Si no sabes
  la respuesta, admitelo.

  {context}

  Pregunta: {question}

  Respuesta:"""
  prompt = PromptTemplate(input_variables=['context', 'question'], template=template)

  hf_pipe= HuggingFacePipeline(pipeline=instruct_pipeline)

  return load_qa_chain(llm=hf_pipe, chain_type='stuff', prompt=prompt, verbose=True)

In [ ]:
qa_chain = build_qa_chain()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from IPython.display import HTML

def answer_question(question):
  similar_docs = get_similar_docs(question, similar_doc_count=2)
  result = qa_chain({"input_documents": similar_docs, "question": question})
  result_html = f"<p><blockquote style=\"font-size:24\">{question}</blockquote></p>"
  result_html += f"<p><blockquote style=\font-size:18px\">{result['output_text']}</blockquote></p>"
  display(HTML(result_html))

In [ ]:
answer_question("En qué meses me he gastado más de $100.000 en Luz?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Debajo hay una instruccion que describe una tarea. Escribe una respuesta que completa apropiadamente la peticion.
  
  Instruccion:
  Eres un asistente personal (chatbot) teniendo una conversacion con un humano y tu trabajo es ayudar a los usuarios suministrandoles la informacion mas acertada y detallada sobre sus documentos basandote en sus preguntas.
  Solo utiliza informacion en los siguientes parrafos para responder la pregunta al final. Explica la respuesta referenciando estos parrafos. Si no sabes
  la respuesta, admitelo.
  
  
  
  Pregunta: En qué meses me he gastado más de $100.000 en Luz?
  
  Respuesta:

> Finished chain.

> Finished chain.


In [ ]:
for doc in get_similar_docs("En qué meses me he gastado más de $100.000 en Luz?", 2):
  print(f"a{doc.page_content}")

In [ ]:
db.similarity_search("En qué meses me he gastado más de $100.000 en Luz?", k=2)

[]